In [1]:
import os
import pyrootutils
root = pyrootutils.setup_root('.', pythonpath=True)
hydra_cfg_path = root / "image_dataset_experiments" / "configs"
os.chdir(root / "image_dataset_experiments")

In [2]:
from hydra import compose, initialize
from pathlib import Path

import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning import LightningModule
import hydra
from omegaconf import DictConfig
import time
import logging

from tqdm import tqdm
from torch.autograd import grad
import functorch as ftorch
import optree

In [3]:
'..' / hydra_cfg_path.relative_to(root)

PosixPath('../image_dataset_experiments/configs')

In [4]:
with initialize(version_base=None, config_path= '../configs'):
    cfg = compose(config_name="tracing", return_hydra_config=True)

# Origin data

In [5]:
dm = hydra.utils.instantiate(cfg.datamodule, data_root=os.environ['PYTORCH_DATASET_ROOT'])
dm.prepare_data()
dm.setup('fit')

Files already downloaded and verified
Files already downloaded and verified


In [55]:
results1 = torch.load('outputs/tracing/bin_cifar10_wrn/2022-12-23_22-49-25/result.pt')
results2 = torch.load('outputs/tracing/bicifar10_wrn_gd/result.pt')


In [63]:
results  = results1

In [64]:
true_error_labels = torch.isin(torch.arange(results.shape[1]), dm.flipped_idx)
score = results.sum(axis=0)
ranking = score.argsort()
print(results.shape)
print(score[ranking])
print(ranking)
num_top = [int(results.shape[1] * i) for i in [0.05, 0.1, 0.15, 0.2]]
acc = []
for t in num_top:
    candidate = ranking[:t]
    first = true_error_labels[candidate].sum() 
    acc.append((first/t).item()*100)
    print(f'{acc[-1]:.4f}', end=';')

(2000, 10000)
[-16783.57544994 -16041.8090775  -15823.83550739 ...   2337.38911104
   2373.66598511   2402.05260062]
[5748 3985 1239 ...  710  538 8285]
99.0000;98.9000;91.0667;72.8000;